[View source on GitHub]: https://github.com/wadmp/wadmp.github.io/blob/master/jupyter_notebooks/create_licence.ipynb
[Notebook Viewer]: https://nbviewer.jupyter.org/github/wadmp/wadmp.github.io/blob/master/jupyter_notebooks/create_licence.ipynb
[Run in binder]: https://mybinder.org/v2/gh/wadmp/wadmp.github.io/master?filepath=jupyter_notebooks%2Fcreate_licence.ipynb
[Run in Google Colab]: https://colab.research.google.com/github/wadmp/wadmp.github.io/blob/master/jupyter_notebooks/create_licence.ipynb

| [![GitHub logo](https://raw.githubusercontent.com/wadmp/wadmp.github.io/master/images/github_logo.png)][View source on GitHub] | [![Jupyter logo](https://raw.githubusercontent.com/wadmp/wadmp.github.io/master/images/jupyter_logo.png)][Notebook Viewer] | [![binder logo](https://raw.githubusercontent.com/wadmp/wadmp.github.io/master/images/binder_logo.png)][Run in binder] | [![Colab logo](https://raw.githubusercontent.com/wadmp/wadmp.github.io/master/images/colab_logo.png)][Run in Google Colab] |
|:---------------------:|:---------------:|:-------------:|:-------------------:|
| [View source on GitHub] | [Notebook Viewer] | [Run in binder] | [Run in Google Colab] |

## Introduction
This notebook provides an example of using the public REST API of WebAccess/DMP.

It allows the user to create a new licence on the system by selecting from the list of available "permissions".

### Requirements
* If you are running in Jupyter Notebook, you MAY find that the "SelectMultiple" widget does not display correctly. If this happens, refresh the browser page after having run the Setup cell.
* If you are running in Jupyter Lab, you will need to install this JupyterLab extension:
  * @jupyter-widgets/jupyterlab-manager
* You need to have an existing user account on the WA/DMP instance.

### Usage
In the "Global Variables" cell below, change BASE_URL to match the particular WA/DMP instance that you are using.

Then run the cells *one at a time* (Shift-Enter).
(For this particular notebook, you *can't* use the 'Run All' command!)

Some cells prompt for user input.
When the "Select required permissions" cell is run, you can select multiple permissions from the list by Ctrl-click'ing.


## Setup
This may take a minute ...

In [ ]:
%%capture

# Install packages in the current Jupyter kernel
import sys
!{sys.executable} -m pip install requests
!{sys.executable} -m pip install ipywidgets
!jupyter nbextension enable --py widgetsnbextension

import requests
import json
import ipywidgets as widgets
from datetime import datetime
import sys

## Global variables

In [ ]:
BASE_URL = 'https://gateway.wadmp.com'
BASE_PATH = 'api'
SESSION = requests.Session()  # Use one HTTPS session for all API calls

PERMISSION_NAMES = [
    "ViewUsers",
    "CreateCompany",
    "ManageRoles",
    "DeleteUser",
    "CreateUser",
    "DeleteCompany",
    "EditCompany",
    "ViewCompany",
    "EditUser",
    "ViewDevice",
    "CreateDevice",
    "ClaimDevice",
    "ViewDMServer",
    "AdmDMServer",
    "ChangeDeviceDMServer",
    "EditDevice",
    "ManageAppStore",
    "ManageAPIClients",
    "ManageSyncEngine",
    "DeleteDevice"
]

## Functions to be used later

In [ ]:
def login(username, password):
    """Login to the system, and return a token
    """
    url = f"{BASE_URL}/public/auth/connect/token"
    credentials = {'username': username, 'password': password, 'client_id': 'python', 'grant_type': 'password'}
    print(f"Sending POST request to {url} with:\n"
          f"    credentials={credentials}\n")
    response = SESSION.post(url, data=credentials)

    print(response.status_code)
    try:
        print(json.dumps(response.json(), indent=4, sort_keys=True))
    except ValueError:
        print(response.text)

    if response.status_code == requests.codes['ok']:
        return response.json()["access_token"]
    else:
        print("Failed to login!")
        sys.exit(1)


def create_licence(permissions, devices=None, expiry=None):
    """Creates a new license key in the system.
    
       'permissions' is a decimal number rpresenting the sum of all individual permissions.
       'devices' is the limit on the number of devices allowed.
       'expiry' is a string representing the date-time on which the licence will expire.
        Use ISO 8601 format and UTC timezone. e.g. "2021-03-18T09:53:51Z"
    """
    url = f"{BASE_URL}/{BASE_PATH}/licenses"
    model = {
      "expiration_date": expiry,
      "devices_allowed": devices,
      "default_user_permissions": permissions
    }
    print(f"\nSending POST request to {url} with:\n"
          f"    model={model}\n")
    response = SESSION.post(url, json=model)

    print(response.status_code)
    try:
        print(json.dumps(response.json(), indent=4, sort_keys=True))
    except ValueError:
        print(response.text)

    if response.status_code == requests.codes['ok']:
        return response.json()['data']['id']
    else:
        print("Failed to create family!")
        return None


def get_companies(name=None):
    """Gets the list of companies in the system.
    """
    url = f"{BASE_URL}/{BASE_PATH}/companies"
    query = {'name': name}
    print(f"Sending GET request to {url} with:\n"
          f"    name={name}\n")
    response = SESSION.get(url, params=query)

    print(response.status_code)
    try:
        print(json.dumps(response.json(), indent=4, sort_keys=True))
    except ValueError:
        print(response.text)

    if response.status_code == requests.codes['ok']:
        return response.json()['data']
    else:
        print("Failed to retrieve the list of companies!")
        return None


def assign_licence(company_id, licence_id):
    """Sets a license key to a company.
    """
    url = f"{BASE_URL}/{BASE_PATH}/companies/{company_id}/license"
    query = {'key': licence_id}
    print(f"Sending PUT request to {url} with:\n"
          f"    key={licence_id}\n")
    response = SESSION.put(url, params=query)

    print(response.status_code)
    try:
        print(json.dumps(response.json(), indent=4, sort_keys=True))
    except ValueError:
        print(response.text)

    if response.status_code == requests.codes['ok']:
        return True
    else:
        print("Failed to assign the licence!")
        return False

## User input

In [ ]:
devices = input("Enter the number of devices allowed:")

print(f"\nCurrent time is {datetime.utcnow().isoformat()}Z")

expiry = input("Enter the expiry date:")

### Select required permissions

In [ ]:
selected_permissions = widgets.SelectMultiple(
    options=PERMISSION_NAMES,
    rows=20,
    disabled=False
)
display(selected_permissions)

Ctrl-click to select from the list displayed, then run the next cell when you are ready ...

In [ ]:
permissions = 0
for selection in selected_permissions.value:
    weight = 2**(PERMISSION_NAMES.index(selection))
    print(f"- {selection} has weight {weight}")
    permissions += weight

binary_string = format(int(permissions), '020b')
print(f"Total = {permissions} decimal (= {binary_string} binary)")

## Login to server

In [ ]:
USERNAME = input("Enter WebAccess/DMP username:")
PASSWORD = input("Enter password:")
user_token = login(USERNAME, PASSWORD)
SESSION.headers.update({'Authorization': f'Bearer {user_token}'})

## Create new licence

In [ ]:
licence_id = create_licence(permissions, devices, expiry)

## Get Company ID

In [ ]:
COMPANY = input("Enter name of company to which you want to assign the licence:")
matching_companies = get_companies(COMPANY)
if len(matching_companies) == 0:
    print("Company name not found!")
    sys.exit(1)
elif len(matching_companies) == 1:
    company_id = matching_companies[0]['id']
    print(f"{COMPANY} has company ID {company_id}")
else:
    print("More than one company matches that name! Just in case, we'll stop here.")
    sys.exit(1)

## Assign licence to company

In [ ]:
assign_licence(company_id, licence_id)